In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.model_selection import train_test_split

"""
    Created by Mohsen Naghipourfar on 3/26/18.
    Email : mn7697np@gmail.com
    Website: http://ce.sharif.edu/~naghipourfar
"""

/Users/Future/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.ge

'\n    Created by Mohsen Naghipourfar on 3/26/18.\n    Email : mn7697np@gmail.com\n    Website: http://ce.sharif.edu/~naghipourfar\n'

In [2]:
# Hyper-Parameters
LEARNING_RATE = 1e-3
DROP_OUT = 0.5
N_SAMPLES = 10787
N_FEATURES = 19671
N_DISEASES = 34
N_BATCHES = 2000
N_EPOCHS = 250
N_BATCH_LEARN = 10
N_RANDOM_FEATURES = 200
neurons = {
    'in': 200,
    'l1': 1024,
    'l2': 512,
    'l3': 256,
    'l4': 128,
    'out': N_DISEASES
}


In [3]:
def modify_output(target):
    global N_DISEASES
    output_dict = {}
    a = target[0].value_counts()
    # print(a)
    a = pd.DataFrame(a)
    i = 0
    for row in a.itertuples():
        output_dict[row[0]] = i
        i += 1
    N_DISEASES = i
    new_output = [[0 for _ in range(N_DISEASES)] for __ in range(N_SAMPLES)]
    for idx, y in target.iterrows():
        new_output[idx][output_dict[y[0]]] = 1
    return new_output

In [21]:
# Load Data
x_data = pd.read_csv("../Data/fpkm_normalized.csv", header=None)
y_data = pd.read_csv("../Data/disease.csv", header=None)
y_data = pd.DataFrame(modify_output(y_data))


In [23]:
# Random Feature Selection
random_feature_indices = np.random.choice(19671, N_RANDOM_FEATURES, replace=False)
x_data = x_data[random_feature_indices]

# Train/Test Split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20)

In [44]:
# x_train.columns = range(x_train.shape[1])
# x_test.columns = range(x_test.shape[1])

False

In [63]:
# Design Model
input_layer = Input(shape=(neurons['in'],))

l1 = Dense(neurons['l1'], activation='relu')(input_layer)

l2 = Dense(neurons['l2'], activation='relu')(l1)

l3 = Dense(neurons['l3'], activation='relu')(l2)

l4 = Dense(neurons['l4'], activation='relu')(l3)

output_layer = Dense(neurons['out'], activation='softmax')(l4)

# Compile Model
network = Model(input_layer, output_layer)

network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              205824    
_________________________________________________________________
dense_32 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_33 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_34 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_35 (Dense)             (None, 34)                4386      
Total params: 899,234
Trainable params: 899,234
Non-trainable params: 0
_________________________________________________________________


/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspe

In [51]:
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [64]:
# # Train Model
network.fit(x=x_train.as_matrix(),
            y=y_train.as_matrix(),
            epochs=N_EPOCHS,
            batch_size=N_BATCHES,
            shuffle=True,
            validation_data=(x_test.as_matrix(), y_test.as_matrix())
            )

# TODO: Use Callback function for our network!

/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspe

Train on 8629 samples, validate on 2158 samples
Epoch 1/250
8629/8629 [==============================] - 0s - loss: 3.4607 - acc: 0.1110 - val_loss: 3.2705 - val_acc: 0.1478
Epoch 2/250
8629/8629 [==============================] - 0s - loss: 3.1448 - acc: 0.1494 - val_loss: 2.9128 - val_acc: 0.2090
Epoch 3/250
8629/8629 [==============================] - 0s - loss: 2.7503 - acc: 0.2332 - val_loss: 2.5014 - val_acc: 0.3299
Epoch 4/250
8629/8629 [==============================] - 0s - loss: 2.3033 - acc: 0.4515 - val_loss: 2.0593 - val_acc: 0.4694
Epoch 5/250
8629/8629 [==============================] - 0s - loss: 1.8619 - acc: 0.5533 - val_loss: 1.6764 - val_acc: 0.5825
Epoch 6/250
8629/8629 [==============================] - 0s - loss: 1.4953 - acc: 0.6626 - val_loss: 1.3784 - val_acc: 0.6719
Epoch 7/250
8629/8629 [==============================] - 0s - loss: 1.2152 - acc: 0.7117 - val_loss: 1.1395 - val_acc: 0.7150
Epoch 8/250
8629/8629 [==============================] - 0s - loss: 1.